In [1]:
import sys
import pandas as pd
sys.path.append('../')
from src.evaluate import *
from src.midi_processing import *

In [2]:
# Importing Chord Vocabulary
chord_df_major = pd.read_csv("../data/chord_vocabulary_major.csv", index_col = 0)
chord_df_minor = pd.read_csv("../data/chord_vocabulary_minor.csv", index_col = 0)
chord_vocab_major, chord_vocab_minor = [], []
for index, name, note_intervals in chord_df_major.itertuples():
    chord_vocab_major.append(Chord(index, name, [int(x) for x in note_intervals.split(',')]))
for index, name, note_intervals in chord_df_minor.itertuples():
    chord_vocab_minor.append(Chord(index, name, [int(x) for x in note_intervals.split(',')]))

In [3]:
# Defining penalties for chord progression
penalties_chord_progression_major = pd.read_csv("../data/chord_progression_major.csv", header = 1, index_col = 0)
penalties_chord_progression_minor = pd.read_csv("../data/chord_progression_minor.csv", header = 1, index_col = 0)
penalties_chord_progression_major = dict(penalties_chord_progression_major.stack())
penalties_chord_progression_minor = dict(penalties_chord_progression_minor.stack())

In [4]:
# Importing Weights
weight_df = pd.read_csv("../data/soft_constraint_weights_temp.csv")

# Defining dictionary of weights for each soft constraint option:
soft_constraint_w_weights = {}
for _, name, w in weight_df.itertuples(): #name population is same as soft_constraint_options
    soft_constraint_w_weights[name] = float(w)
assert sum(v for v in soft_constraint_w_weights.values() if v > 0) == 100

In [5]:
# Defining dictionary of hard and soft constraint options:
hard_constraint_options = ['musical input', 'voice range', 'chord membership', 'first last chords',
                           'chord repetition', 'chord bass repetition', 'adjacent bar chords', 'voice crossing', 'parallel movement',
                          'chord spacing', 'incomplete chord', 'chord distribution']
soft_constraint_options = ['chord progression', 'chord repetition', 'chord bass repetition', 'leap resolution',
                           'melodic movement', 'note repetition', 'parallel movement', 'voice overlap', 'adjacent bar chords',
                           'chord spacing', 'distinct notes', 'incomplete chord', 'voice crossing', 'voice range',
                           'second inversion', 'first inversion', 'chord distribution']

In [6]:
# Defining which hard constraints to use
hard_constraints = {x: True if x in ['musical input', 'voice range', 'chord membership', 'first last chords',
                                     'voice crossing', 'parallel movement',
                                     'chord spacing', 'incomplete chord'] else False for x in hard_constraint_options}

In [7]:
# Test Case Characteristics
key = 9
tonality = 'major'
meter = 4
first_on_beat = 2
mode = 'D'

In [8]:
if tonality == 'major':
    penalties_chord_progression = penalties_chord_progression_major
    chord_vocab = chord_vocab_major
else:
    penalties_chord_progression = penalties_chord_progression_minor
    chord_vocab = chord_vocab_minor 

In [9]:
# Importing Test Case
test_case = pd.read_csv("../data/Ach_bleib'_bei_uns_Herr_Jesu_Christ_(part)_transcription.csv", skiprows = 5, header = None, index_col = 0)
list_x = list(test_case.iloc[:-1,-1].apply(lambda y: y.split(', ')))
list_x = [[int(x) for x in i] for i in list_x]
list_c_names = test_case.loc['Chords'].apply(lambda y: y.replace('\'','').split(', ')).iloc[0]
list_c = []
for chord_name in list_c_names:
    exists = False
    for chord in chord_vocab:
        if chord.name == chord_name:
            exists = True
            list_c.append(chord.index)
            break
    if not exists:
        list_c.append(-1)

In [10]:
bach_cost = evaluate_cost(list_x, list_c, key, tonality, meter, first_on_beat, mode,
                  chord_vocab,
                  penalties_chord_progression,
                  hard_constraints, 1000,
                  soft_constraint_w_weights)

In [11]:
bach_cost_chord_membership = evaluate_cost([x[:5]+x[6:] for x in list_x], list_c[:5]+list_c[6:], key, tonality, meter, first_on_beat, mode,
                  chord_vocab,
                  penalties_chord_progression,
                  {k: True if k == 'chord membership' else False for k in hard_constraint_options}, 1000,
                  soft_constraint_w_weights)

In [12]:
ignore = ['hard constraint chord membership', 'hard constraint chord spacing', 'hard constraint incomplete chord', 'soft constraint chord progression']
total_bach_cost = sum(v for k, v in bach_cost.items() if k not in ignore) + sum(v for k, v in bach_cost_chord_membership.items() if k in ignore)
print(total_bach_cost)

216.0


In [13]:
#bach_cost

In [14]:
#bach_cost_chord_membership

In [15]:
#array_to_midi(list_x, [20]*4, beat = 600, dest_file_path = '../outputs/bach_Ach bleib\' bei unsm Herr Jesu Christ (part).mid', held_notes = False, offset = 3)